Define your variables (That's the only cells you need to change). Then run the whole notebook

In [ ]:
### files metadata
xy_pixel: 0.0764616
z_pixel: 0.4

image_path = '/usr/home/mpg08/aicha.hajiali/TLI_project/TLI_data/registered/211104_analysis/4D_GFP_GFP_SyNRA_4D_211104_brain1.2.tif'
save_path = '/usr/home/mpg08/aicha.hajiali/TLI_project/TLI_data/registered/211104_analysis/'
save_name = '4D_GFP_211104_brain1.2'

#N2V parameters
model_name ='n2v_3D_v6'
model_path ='/home/mpg08/aicha.hajiali/TLI_project/preprocessing/n2v/models/'

#Clahe parameters:
clipLimit = 1
kernel_size = (45, 45)

In [ ]:
if save_path[-1] != '/':
    save_path += '/'

Load needed packages

In [ ]:
import tifffile as tif
import utils.denoise as denoise
import utils.datautils as datautils
import Neurosetta as neu

In [ ]:
# Read 4D GFP image
image_4D = tif.imread(image_path)

In [ ]:
image_4D = denoise.N2V_4D(image_4D, model_name=model_name, model_path=model_path, 
                            save=True, save_path=save_path, xy_pixel=0.076, 
                            z_pixel=0.4, save_file=save_name) 

In [ ]:
image_4D = denoise.clahe_4D(image_4D, kernel_size=kernel_size, clipLimit=clipLimit, 
                            save=True, save_path=save_path, xy_pixel=0.076, 
                            z_pixel=0.4, save_file=save_name) 